# Оценка Retrieval с использованием LLM

Этот ноутбук выполняет оценку качества поискового движка:
- Загружает запросы из `queries.json`
- Выполняет поиск топ-5 документов для каждого запроса
- Использует LLM для оценки релевантности каждого документа
- Считает **Recall@5** и **Precision@5**

## 1. Импорты и настройки

In [15]:
import json
import logging
import sys
from pathlib import Path

# Добавляем путь к проекту
project_root = Path("/srv/nlp1/eval_dir/T-bank_NLP_")
sys.path.insert(0, str(project_root))

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

print(f"📁 Корневая директория проекта: {project_root}")

📁 Корневая директория проекта: /srv/nlp1/eval_dir/T-bank_NLP_


In [16]:
from tqdm import tqdm

from tplexity.retriever.retriever_service import RetrieverService

print("✅ Импорты выполнены успешно")

✅ Импорты выполнены успешно


## 2. Инициализация сервисов

In [17]:
# Инициализация RetrieverService
print("🔄 Инициализация RetrieverService...")
retriever = RetrieverService()
print("✅ RetrieverService инициализирован")

2025-11-16 18:48:37,136 - INFO - 🔄 [retriever_service] Инициализация гибридного поисковика
2025-11-16 18:48:37,138 - INFO - 🔄 [retriever][vector_search] Инициализация клиента Qdrant


🔄 Инициализация RetrieverService...


2025-11-16 18:48:37,405 - INFO - HTTP Request: GET https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
2025-11-16 18:48:37,406 - INFO - ✅ [retriever][vector_search] Клиент Qdrant инициализирован: 8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333
2025-11-16 18:48:37,406 - INFO - ✅ [retriever][vector_search] Dense модель инициализирована, размерность: 1536
2025-11-16 18:48:37,407 - INFO - ✅ [retriever][vector_search] BM25 модель инициализирована


✅ RetrieverService инициализирован


In [18]:
# Инициализация LLM (используем Qwen как наиболее доступный)
print("🔄 Инициализация LLM...")
llm_client = get_llm("qwen")
print(f"✅ LLM инициализирован: model={llm_client.model}")

🔄 Инициализация LLM...
✅ LLM инициализирован: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ


## 3. Загрузка запросов

In [19]:
# Загружаем queries
queries_path = project_root / "src/eval/eval_data/queries.json"

with open(queries_path, encoding="utf-8") as f:
    all_queries = json.load(f)

print(f"📊 Загружено {len(all_queries)} запросов")

# Для тестирования можно взять подмножество
NUM_QUERIES = 50  # Измените на None для всех запросов
queries = all_queries[:NUM_QUERIES] if NUM_QUERIES else all_queries

print(f"✅ Будет обработано {len(queries)} запросов")

📊 Загружено 4991 запросов
✅ Будет обработано 50 запросов


## 4. Функция оценки релевантности через LLM

In [ ]:
async def judge_relevance(query: str, document: str, llm_client) -> bool:
    """
    Оценивает релевантность документа запросу через LLM

    Args:
        query: Поисковый запрос
        document: Текст документа
        llm_client: Клиент LLM

    Returns:
        bool: True если документ релевантен, False иначе
    """
    prompt = f"""Ты - эксперт по оценке релевантности документов поисковым запросам.

Запрос пользователя: {query}

Документ: {document}

Задача: Определи, релевантен ли данный документ запросу пользователя.
Документ считается релевантным, если он содержит информацию, которая помогает ответить на запрос или относится к его теме.

Ответь ТОЛЬКО одним словом: "ДА" если документ релевантен, "НЕТ" если не релевантен."""

    messages = [
        {"role": "system", "content": "Ты - эксперт по оценке релевантности документов."},
        {"role": "user", "content": prompt},
    ]

    try:
        response = await llm_client.generate(
            messages=messages,
            temperature=0.0,  # Детерминированный ответ
            max_tokens=10,
        )

        # Проверяем ответ
        response = response.strip().upper()
        return "ДА" in response or "YES" in response

    except Exception as e:
        logger.error(f"Ошибка при вызове LLM: {e}")
        return False


print("✅ Функция judge_relevance определена")

✅ Функция judge_relevance определена


## 5. Функция оценки одного запроса

In [ ]:
async def evaluate_query(query_data: dict, retriever, llm_client, top_k: int = 5) -> dict:
    """
    Оценивает качество поиска для одного запроса

    Args:
        query_data: Данные запроса из queries.json
        retriever: RetrieverService
        llm_client: Клиент LLM
        top_k: Количество документов для оценки

    Returns:
        dict: Результаты оценки
    """
    query_text = query_data["query"]
    ground_truth_id = f"{query_data['id_channel']}_{query_data['id_message']}"

    # Выполняем поиск
    try:
        search_results = await retriever.search(query=query_text, top_k=top_k, top_n=top_k, use_rerank=False)
    except Exception as e:
        logger.error(f"Ошибка при поиске: {e}")
        return {
            "query": query_text,
            "ground_truth_id": ground_truth_id,
            "error": str(e),
            "recall": 0.0,
            "precision": 0.0,
        }

    if not search_results:
        return {
            "query": query_text,
            "ground_truth_id": ground_truth_id,
            "recall": 0.0,
            "precision": 0.0,
            "relevant_count": 0,
            "found_ground_truth": False,
        }

    # Проверяем каждый документ на релевантность через LLM
    relevance_judgments = []
    found_ground_truth = False

    for _doc_id, score, text, metadata in search_results:
        # Проверяем, это ли ground truth документ
        doc_id_from_metadata = metadata.get("doc_id", "") if metadata else ""
        is_ground_truth = doc_id_from_metadata == ground_truth_id

        if is_ground_truth:
            found_ground_truth = True

        # Оцениваем релевантность через LLM
        is_relevant = await judge_relevance(query_text, text, llm_client)

        relevance_judgments.append(
            {
                "doc_id": doc_id_from_metadata,
                "is_ground_truth": is_ground_truth,
                "is_relevant_by_llm": is_relevant,
                "score": score,
                "text_preview": text[:200],
            }
        )

    # Подсчет метрик
    relevant_count = sum(1 for j in relevance_judgments if j["is_relevant_by_llm"])
    print("РЕЛЕВАНТНЫХ В ЗАПРОСЕ:", relevant_count)

    # Recall@K = 1 если нашли ground truth, иначе 0 (так как у нас только 1 релевантный)
    recall = 1.0 if found_ground_truth else 0.0

    # Precision@K = (количество релевантных по мнению LLM) / K
    precision = relevant_count / top_k if top_k > 0 else 0.0

    return {
        "query": query_text,
        "ground_truth_id": ground_truth_id,
        "recall": recall,
        "precision": precision,
        "relevant_count": relevant_count,
        "found_ground_truth": found_ground_truth,
        "judgments": relevance_judgments,
    }


print("✅ Функция evaluate_query определена")

✅ Функция evaluate_query определена


## 6. Запуск оценки

In [ ]:
# Конфигурация
TOP_K = 5

print(f"🚀 Начало оценки для {len(queries)} запросов (top_k={TOP_K})")
print("=" * 80)

# Списки для хранения результатов
all_results = []
all_recalls = []
all_precisions = []

# Обрабатываем каждый запрос
for i, query_data in enumerate(tqdm(queries, desc="Оценка запросов")):
    result = await evaluate_query(query_data, retriever, llm_client, top_k=TOP_K)

    all_results.append(result)
    all_recalls.append(result["recall"])
    all_precisions.append(result["precision"])

    # Логируем каждый 5-й результат
    if (i + 1) % 5 == 0:
        print(f"\n[{i + 1}/{len(queries)}] Query: {result['query'][:60]}...")
        print(f"  Recall@{TOP_K}: {result['recall']:.2f}, Precision@{TOP_K}: {result['precision']:.2f}")
        print(
            f"  Found ground truth: {result['found_ground_truth']}, Relevant docs: {result['relevant_count']}/{TOP_K}"
        )

print("\n" + "=" * 80)
print("✅ Оценка завершена!")

🚀 Начало оценки для 50 запросов (top_k=5)


2025-11-16 18:48:59,451 - INFO - 🔍 [retriever_service] Начало поиска для запроса: Запущено ли сообщество для инвесторов в приложении...

Batches: 100%|██████████| 1/1 [00:00<00:00, 47.65it/s]
2025-11-16 18:48:59,740 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/query "HTTP/1.1 200 OK"
2025-11-16 18:48:59,742 - INFO - ✅ [retriever_service] Гибридный поиск завершен, найдено результатов: 5
2025-11-16 18:48:59,742 - INFO - ✅ [retriever_service] Поиск завершен, возвращено 5 результатов
2025-11-16 18:48:59,811 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:48:59,813 - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-16 18:49:00,042 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:49:00,044 - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-16 18:49:00,672 - INFO - HTTP Request:

РЕЛЕВАНТНЫХ В ЗАПРОСЕ: 3



Batches: 100%|██████████| 1/1 [00:00<00:00, 48.72it/s]
2025-11-16 18:49:00,987 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/query "HTTP/1.1 200 OK"
2025-11-16 18:49:00,989 - INFO - ✅ [retriever_service] Гибридный поиск завершен, найдено результатов: 5
2025-11-16 18:49:00,989 - INFO - ✅ [retriever_service] Поиск завершен, возвращено 5 результатов
2025-11-16 18:49:01,092 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:49:01,093 - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-16 18:49:01,194 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:49:01,195 - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-16 18:49:01,294 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:49:01,296 - INFO - ✅ [llm_client] Ответ получен от 

РЕЛЕВАНТНЫХ В ЗАПРОСЕ: 2


...

РЕЛЕВАНТНЫХ В ЗАПРОСЕ: 1



Batches: 100%|██████████| 1/1 [00:00<00:00, 52.29it/s]
2025-11-16 18:49:26,295 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/query "HTTP/1.1 200 OK"
2025-11-16 18:49:26,296 - INFO - ✅ [retriever_service] Гибридный поиск завершен, найдено результатов: 5
2025-11-16 18:49:26,297 - INFO - ✅ [retriever_service] Поиск завершен, возвращено 5 результатов
2025-11-16 18:49:26,357 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:49:26,358 - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-16 18:49:26,421 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:49:26,421 - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-16 18:49:26,492 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:49:26,493 - INFO - ✅ [llm_client] Ответ получен от 

РЕЛЕВАНТНЫХ В ЗАПРОСЕ: 4



Batches: 100%|██████████| 1/1 [00:00<00:00, 50.32it/s]
2025-11-16 18:49:26,778 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/query "HTTP/1.1 200 OK"
2025-11-16 18:49:26,779 - INFO - ✅ [retriever_service] Гибридный поиск завершен, найдено результатов: 5
2025-11-16 18:49:26,780 - INFO - ✅ [retriever_service] Поиск завершен, возвращено 5 результатов
2025-11-16 18:49:26,837 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:49:26,838 - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-16 18:49:26,899 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:49:26,900 - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-16 18:49:26,956 - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 18:49:26,957 - INFO - ✅ [llm_client] Ответ получен от 

РЕЛЕВАНТНЫХ В ЗАПРОСЕ: 4

[50/50] Query: Почему розничные инвесторы в России всё чаще получают доступ...
  Recall@5: 0.00, Precision@5: 0.80
  Found ground truth: False, Relevant docs: 4/5

✅ Оценка завершена!


## 7. Вычисление итоговых метрик

In [ ]:
# Вычисляем средние метрики
avg_recall = sum(all_recalls) / len(all_recalls) if all_recalls else 0.0
avg_precision = sum(all_precisions) / len(all_precisions) if all_precisions else 0.0

# Выводим результаты
print("=" * 80)
print("📊 ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("=" * 80)
print(f"Количество запросов: {len(queries)}")
print(f"Top-K: {TOP_K}")
print("\n🎯 Метрики:")
print(f"  Recall@{TOP_K}:    {avg_recall:.4f} ({avg_recall * 100:.2f}%)")
print(f"  Precision@{TOP_K}: {avg_precision:.4f} ({avg_precision * 100:.2f}%)")
print("\n📝 Интерпретация:")
print(f"  - Recall@{TOP_K} показывает, в скольких случаях ground truth документ попал в топ-{TOP_K}")
print(f"  - Precision@{TOP_K} показывает, какая доля найденных документов релевантна (по мнению LLM)")
print("=" * 80)

## 8. Детальный анализ результатов

In [ ]:
# Статистика по ground truth
found_gt_count = sum(1 for r in all_results if r.get("found_ground_truth", False))
print(f"Ground truth документ найден: {found_gt_count}/{len(queries)} ({found_gt_count / len(queries) * 100:.1f}%)")

# Распределение количества релевантных документов
from collections import Counter

relevant_counts = [r.get("relevant_count", 0) for r in all_results]
distribution = Counter(relevant_counts)

print(f"\nРаспределение количества релевантных документов в топ-{TOP_K}:")
for count in sorted(distribution.keys()):
    freq = distribution[count]
    print(f"  {count} релевантных: {freq} запросов ({freq / len(queries) * 100:.1f}%)")

## 9. Примеры результатов

In [ ]:
# Показываем несколько примеров
print("\n" + "=" * 80)
print("📋 ПРИМЕРЫ РЕЗУЛЬТАТОВ")
print("=" * 80)

for i, result in enumerate(all_results[:3]):
    print(f"\n[Пример {i + 1}]")
    print(f"Query: {result['query'][:100]}...")
    print(f"Ground truth found: {result.get('found_ground_truth', False)}")
    print(f"Recall@{TOP_K}: {result['recall']:.2f}, Precision@{TOP_K}: {result['precision']:.2f}")
    print("\nНайденные документы:")

    for j, judgment in enumerate(result.get("judgments", []), 1):
        is_gt = "[GT]" if judgment["is_ground_truth"] else "    "
        is_rel = "✓" if judgment["is_relevant_by_llm"] else "✗"
        print(f"  {is_gt} {j}. {is_rel} Score: {judgment['score']:.3f}")
        print(f"       {judgment['text_preview'][:80]}...")

    print("-" * 80)

## 10. Сохранение результатов

In [ ]:
# Сохраняем результаты в JSON
output_data = {
    "config": {"num_queries": len(queries), "top_k": TOP_K, "llm_model": llm_client.model},
    "metrics": {"recall@5": avg_recall, "precision@5": avg_precision},
    "detailed_results": all_results,
}

output_path = project_root / "src/eval/eval_results_llm.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print(f"✅ Результаты сохранены в: {output_path}")